In [ ]:
import os
import pandas as pd
from keras.models import load_model
import MalGAN_utils
import MalGAN_gen_adv_examples

# 입/출력 경로를 지정한다.
save_path = "./MalGAN_output"
model_path = "./MalGAN_input/malconv.h5"
log_path = "./MalGAN_output/adversarial_log.csv"
pad_percent = 0.1
threshold = 0.6
step_size = 0.01
limit = 0.
input_samples = "./MalGAN_input/sampleIn.csv"

# 적대적 샘플 생성에 GPU 사용여부를 설정한다.
MalGAN_utils.limit_gpu_memory(limit)

# 샘플의 이름과 레이블이 저장된 csv 파일을 읽어 데이터프레임에 저장한다.
df = pd.read_csv(input_samples, header=None)
fn_list = df[0].values

# 미리 계산된 MalConv 모델을 불러온다.
model = load_model(model_path)

# 고속 경사 부호법을 사용해 적대적 악성코드를 생성한다.
adv_samples, log = MalGAN_gen_adv_examples.gen_adv_samples(
    model, fn_list, pad_percent, step_size, threshold
)

# 결과 로그를 저장하고 샘플을 디스크에 저장한다.
log.save(log_path)
for fn, adv in zip(fn_list, adv_samples):
    _fn = fn.split('/')[-1]
    dst = os.path.join(save_path, _fn)
    print(dst)
    with open(dst, 'wb') as f:
        f.write(adv)